In [51]:
import random
import logging
from collections import namedtuple, Counter
from functools import reduce

In [52]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1)
             for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [53]:
def tournament(population, tournament_size=2):
   return max(random.choices(population, k=tournament_size), key=lambda i: i.fitness)

def evaluate(genome, P):
    solution = list()
    covering= set()
    [solution.append(P[i]) for i, n in enumerate(genome) if n == 1]
    covering = reduce(lambda a, b: set(a) | set(b), solution, covering)
    return len(covering), -sum(len(_) for _ in solution)

def covering(genome):
   s = set()
   for i in genome:
      s = s.union(set(i))
   return len(s)

def mutation(g, len):
   point = random.randint(0, len - 1)
   return g[:point] + (1 - g[point],) + g[point + 1:]

def cross_over(g1, g2, len):
   cut = random.randint(0, len)
   return g1[:cut] + g2[cut:]

def create_population(prob_size, pop_size, P):
   population = list()
   for i in [tuple([0 for _ in range(prob_size)]) for _ in range(pop_size)]:
      population.append(Individual(i, evaluate(i, P)))
   return population

In [54]:
def genetic(N, P, tournament_size, pop_size, mutation_rate):
   prob_size = len(P)
   population = create_population(prob_size, pop_size, P)
   num_gen = 0

   for g in range(GEN):
      num_gen += 1
      offspring = list()
      for i in range(OFFSPRING_SIZE):
         if random.random() < mutation_rate:
            p = tournament(population, tournament_size=tournament_size)
            o = mutation(p.genome, prob_size)
         else:
            p1 = tournament(population, tournament_size=tournament_size)
            p2 = tournament(population, tournament_size=tournament_size)
            o = cross_over(p1.genome, p2.genome, prob_size)
      f = evaluate(o, P)
      offspring.append(Individual(o, f))
      population += offspring
      population = sorted(population, key=lambda i: i.fitness, reverse=True)[:pop_size]

      if population[0].fitness[0] == N:
         return population, num_gen


In [55]:
logging.getLogger().setLevel(logging.INFO)

Individual = namedtuple('Individual', ('genome', 'fitness'))
POP_SIZE = 100
GEN = 500
OFFSPRING_SIZE = 100
SEED = 42
MUTATION_RATE = 0.5

for N in [
   5,
   10,
   20,
   100,
   500,
   1000
]:
   population, num_gen = genetic(N, problem(N, SEED), 2, POP_SIZE, MUTATION_RATE)
   w = -population[0].fitness[1]
   logging.info(f"#### N = {N} ####\n" + f"w={w}, num_gen={num_gen}, bloat={int((w-N)*100/N)}%\n")



INFO:root:#### N = 5 ####
w=7, num_gen=141, bloat=40%

INFO:root:#### N = 10 ####
w=21, num_gen=194, bloat=110%

INFO:root:#### N = 20 ####
w=41, num_gen=165, bloat=105%

INFO:root:#### N = 100 ####
w=330, num_gen=336, bloat=230%

INFO:root:#### N = 500 ####
w=2028, num_gen=450, bloat=305%

INFO:root:#### N = 1000 ####
w=4839, num_gen=404, bloat=383%

